# Task-3
## Cat / Dog classifier using transfer learning technique
### 1. Use ConvNet as feature extractor
### 2. Fine-tune ConvNet

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms
from tqdm import tqdm

from utils.mobilenet import MobileNetV2
from torchvision.models import resnet18

from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# ConvNet as feature extractor

## 1. Define transforms

In [2]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

## 2. Build train / test dataloaders

In [3]:
batch_size = 64

dataset_train = datasets.ImageFolder('./data/small/train/', transform=transform)
dataset_val = datasets.ImageFolder('./data/small/validation/', transform=transform)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

## 3. Load MobileNetV2 (with no last layer) and pretrained on ResNet weights

In [4]:
model = MobileNetV2().cuda()
model.load_state_dict(torch.load('utils/mobilenet_v2.pth.tar'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## 4. Train / Val inference to extract features 

In [5]:
X_train, X_test, y_train, y_test = [], [], [], []
with torch.no_grad():
    for data, target in train_loader:
        X_train.extend(model(data.cuda()).cpu().numpy())
        y_train.extend(target.numpy())
        
    for data, target in val_loader:
        X_test.extend(model(data.cuda()).cpu().numpy())
        y_test.extend(target.numpy())

X_train = np.vstack(X_train)
X_test = np.vstack(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

## 5. Use different ML methods for prediction

## KNN

In [6]:
model = KNeighborsClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.74      0.86      0.80       128
        dogs       0.83      0.70      0.76       128

   micro avg       0.78      0.78      0.78       256
   macro avg       0.79      0.78      0.78       256
weighted avg       0.79      0.78      0.78       256



## SVM

In [7]:
model = SVC(gamma='auto').fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.61      0.80      0.69       128
        dogs       0.71      0.48      0.58       128

   micro avg       0.64      0.64      0.64       256
   macro avg       0.66      0.64      0.64       256
weighted avg       0.66      0.64      0.64       256



## Decision Tree

In [8]:
model = DecisionTreeClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.63      0.60      0.62       128
        dogs       0.62      0.65      0.63       128

   micro avg       0.62      0.62      0.62       256
   macro avg       0.63      0.62      0.62       256
weighted avg       0.63      0.62      0.62       256



## Random Forest

In [10]:
model = RandomForestClassifier(n_estimators=500).fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.76      0.74      0.75       128
        dogs       0.75      0.77      0.76       128

   micro avg       0.75      0.75      0.75       256
   macro avg       0.75      0.75      0.75       256
weighted avg       0.75      0.75      0.75       256



## Gradient Boosting

In [11]:
model = GradientBoostingClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.68      0.72      0.70       128
        dogs       0.70      0.66      0.68       128

   micro avg       0.69      0.69      0.69       256
   macro avg       0.69      0.69      0.69       256
weighted avg       0.69      0.69      0.69       256



## Multilayer Perceptron

In [12]:
model = MLPClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.87      0.87      0.87       128
        dogs       0.87      0.88      0.87       128

   micro avg       0.87      0.87      0.87       256
   macro avg       0.87      0.87      0.87       256
weighted avg       0.87      0.87      0.87       256



## 6. Add classification layer to MobileNetV2

In [5]:
class MobileNetV2_CatDog(nn.Module):
    def __init__(self, model):
        super(MobileNetV2_CatDog, self).__init__()
        self.model = model
        self.linear = nn.Linear(model.classifier[1].in_features, 2)
        
    def forward(self, x, extract=False):
        x = self.model(x)
        if extract:
            return x
        x = F.softmax(self.linear(x), dim=1)
        return x

## 7. Train / val

In [13]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx + 1) % len(train_loader) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx , len(train_loader),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## 8. Define MobileNetV2 for Cat / Dog classifier

In [24]:
model = MobileNetV2().cuda()
model.load_state_dict(torch.load('utils/mobilenet_v2.pth.tar'))

model = MobileNetV2_CatDog(model).cuda()

optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

## 9. Train new network

In [33]:
for epoch in range(1, 26):
    train(model, train_loader, optimizer, epoch)
    test(model, val_loader)

Train Epoch: 1 [7/8 (88%)]	Loss: 0.752505
Test set: Accuracy: 170/256 (66.41%)

Train Epoch: 2 [7/8 (88%)]	Loss: 0.713997
Test set: Accuracy: 174/256 (67.97%)

Train Epoch: 3 [7/8 (88%)]	Loss: 0.710280
Test set: Accuracy: 179/256 (69.92%)

Train Epoch: 4 [7/8 (88%)]	Loss: 0.716692
Test set: Accuracy: 180/256 (70.31%)

Train Epoch: 5 [7/8 (88%)]	Loss: 0.700664
Test set: Accuracy: 184/256 (71.88%)

Train Epoch: 6 [7/8 (88%)]	Loss: 0.715826
Test set: Accuracy: 183/256 (71.48%)

Train Epoch: 7 [7/8 (88%)]	Loss: 0.687677
Test set: Accuracy: 183/256 (71.48%)

Train Epoch: 8 [7/8 (88%)]	Loss: 0.706599
Test set: Accuracy: 185/256 (72.27%)

Train Epoch: 9 [7/8 (88%)]	Loss: 0.708344
Test set: Accuracy: 186/256 (72.66%)

Train Epoch: 10 [7/8 (88%)]	Loss: 0.703584
Test set: Accuracy: 186/256 (72.66%)

Train Epoch: 11 [7/8 (88%)]	Loss: 0.692964
Test set: Accuracy: 188/256 (73.44%)

Train Epoch: 12 [7/8 (88%)]	Loss: 0.697720
Test set: Accuracy: 188/256 (73.44%)

Train Epoch: 13 [7/8 (88%)]	Loss: 0.7

## 10. Extract features for ML methods

In [34]:
X_train, X_test, y_train, y_test = [], [], [], []
with torch.no_grad():
    for data, target in train_loader:
        X_train.extend(model(data.cuda(), extract=True).cpu().numpy())
        y_train.extend(target.numpy())
        
    for data, target in val_loader:
        X_test.extend(model(data.cuda(), extract=True).cpu().numpy())
        y_test.extend(target.numpy())

X_train = np.vstack(X_train)
X_test = np.vstack(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

## KNN

In [44]:
model = KNeighborsClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.98      0.98      0.98       128
        dogs       0.98      0.98      0.98       128

   micro avg       0.98      0.98      0.98       256
   macro avg       0.98      0.98      0.98       256
weighted avg       0.98      0.98      0.98       256



## SVM

In [45]:
model = SVC(gamma='auto').fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       1.00      0.66      0.80       128
        dogs       0.75      1.00      0.86       128

   micro avg       0.83      0.83      0.83       256
   macro avg       0.87      0.83      0.83       256
weighted avg       0.87      0.83      0.83       256



## Decision Tree

In [46]:
model = DecisionTreeClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.93      0.95      0.94       128
        dogs       0.95      0.93      0.94       128

   micro avg       0.94      0.94      0.94       256
   macro avg       0.94      0.94      0.94       256
weighted avg       0.94      0.94      0.94       256



## Random Forest

In [47]:
model = RandomForestClassifier(n_estimators=2500).fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.98      0.96      0.97       128
        dogs       0.96      0.98      0.97       128

   micro avg       0.97      0.97      0.97       256
   macro avg       0.97      0.97      0.97       256
weighted avg       0.97      0.97      0.97       256



## Gradient Boosting

In [48]:
model = GradientBoostingClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.98      0.95      0.96       128
        dogs       0.95      0.98      0.97       128

   micro avg       0.96      0.96      0.96       256
   macro avg       0.97      0.96      0.96       256
weighted avg       0.97      0.96      0.96       256



## Multilayer Perceptron

In [49]:
model = MLPClassifier().fit(X_train, y_train) 
print(classification_report(y_test, model.predict(X_test), target_names=['cats', 'dogs']))

              precision    recall  f1-score   support

        cats       0.96      0.98      0.97       128
        dogs       0.98      0.96      0.97       128

   micro avg       0.97      0.97      0.97       256
   macro avg       0.97      0.97      0.97       256
weighted avg       0.97      0.97      0.97       256



# F1-score

https://docs.google.com/spreadsheets/d/17x7mJ764ID8LCqPst4XrE0PfI7VYJtKBtAJnYxIBveE/edit?usp=sharing

|  method | feature extractor | fune-tune |
| --- | --- | --- |
|  KNN | 0,78 | **0,98** |
|  SVM | 0,64 | 0,83 |
|  Decision Tree | 0,62 | 0,94 |
|  Random Forest | 0,75 | 0,97 |
|  Gradient Boosting | 0,69 | 0,96 |
|  MLP | **0,87** | 0,97 |